In [3]:
import pandas as pd
import numpy as np
import requests

# from tqdm.auto import tqdm, trange
from tqdm import tqdm, trange
# tqdm.pandas()


from credentials import bosa_mapping_url

from zipfile import ZipFile
import json
import random
import time
import os, urllib

import glob


In [4]:
def download_if_nexist(url, filename):
    """
    If the (local) file <filename> does not exists, download it from <url>

    Parameters
    ----------
    url: str
       url to fetch
    filename: str
       local file to save

    Returns
    -------

    None
    """
    if not os.path.isfile(filename):
        #gcontext = ssl.SSLContext()
        with urllib.request.urlopen(url) as response:
            with open(filename, "wb") as f:
                f.write(response.read())

In [5]:
datadir = "data/geocoding/"
os.makedirs(datadir, exist_ok=True)
os.makedirs(f"{datadir}/full", exist_ok=True)


In [6]:
sample_size=1000

seed=314

In [7]:
datasets = ["kbo", "rep", "best", "rrn", "resto"]
datasets = ["rrn"]

# Best

In [8]:
if "best" in datasets:
    best_vlg_fn = f"{datadir}/full/openaddress-bevlg.zip"
    download_if_nexist("https://opendata.bosa.be/download/best/openaddress-bevlg.zip", best_vlg_fn)

    best_wal_fn = f"{datadir}/full/openaddress-bewal.zip"
    download_if_nexist("https://opendata.bosa.be/download/best/openaddress-bewal.zip", best_wal_fn)

    best_bru_fn = f"{datadir}/full/openaddress-bebru.zip"
    download_if_nexist("https://opendata.bosa.be/download/best/openaddress-bebru.zip", best_bru_fn)

In [9]:
if "best" in datasets:
    best_full = pd.concat([pd.read_csv(fn, usecols=["municipality_name_de", "municipality_name_fr", "municipality_name_nl", 
                                                    "streetname_de", "streetname_fr", "streetname_nl",
                                                    "postcode", "house_number", "region_code"], dtype=str) for fn in [best_vlg_fn, best_wal_fn, best_bru_fn] ])

In [10]:
if "best" in datasets:
    best_full["street"] = np.where(best_full.region_code == "BE-VLG", best_full.streetname_nl.fillna(best_full.streetname_fr).fillna(best_full.streetname_de),
                          np.where(best_full.region_code == "BE-WAL", best_full.streetname_fr.fillna(best_full.streetname_de).fillna(best_full.streetname_nl),
                          np.where(best_full.region_code == "BE-BRU", best_full.streetname_fr.fillna(best_full.streetname_nl).fillna(best_full.streetname_de), None)))

    best_full["city"] =   np.where(best_full.region_code == "BE-VLG", best_full.municipality_name_nl.fillna(best_full.municipality_name_fr).fillna(best_full.municipality_name_de),
                          np.where(best_full.region_code == "BE-WAL", best_full.municipality_name_fr.fillna(best_full.municipality_name_de).fillna(best_full.municipality_name_nl),
                          np.where(best_full.region_code == "BE-BRU", best_full.municipality_name_fr.fillna(best_full.municipality_name_nl).fillna(best_full.municipality_name_de), None)))

In [11]:
if "best" in datasets:
    best_sample =  best_full[["street", "house_number", "postcode", "city"]].rename(columns={"house_number": "housenumber"})
    best_sample = best_sample.drop_duplicates().sample(sample_size, random_state=seed).reset_index(drop=True)
    best_sample.to_csv(f"{datadir}/best_{sample_size}.csv.gz", index=False)

# KBO


In [12]:
kbo_fn = f"{datadir}/full/kbo_full.zip"
# To be downloaded from https://kbopub.economie.fgov.be/kbo-open-data/

#download_if_nexist("https://kbopub.economie.fgov.be/kbo-open-data/affiliation/xml/files/KboOpenData_0111_2023_05_Full.zip", kbo_fn)


In [13]:
if "kbo" in datasets:
    with ZipFile(kbo_fn) as z:
       # open the csv file in the dataset
        with z.open("address.csv") as f:

            kbo_full = pd.read_csv(f, usecols=["CountryFR", "Zipcode", 
                                               "MunicipalityNL","MunicipalityFR", 
                                               "StreetNL", "StreetFR", "HouseNumber"],
                                dtype=str)
    

In [14]:
if "kbo" in datasets:

    kbo_full = kbo_full[kbo_full.CountryFR.isnull()].copy()
    
    kbo_sample = kbo_full.drop_duplicates(subset=["Zipcode","StreetNL", "StreetFR", "HouseNumber"]).sample(sample_size, random_state=seed)

In [15]:
# kbo_sample

In [16]:
if "kbo" in datasets:
    kbo_sample["lg"] = np.where(kbo_sample.Zipcode.str[0].isin(["2", "3", "8", "9"]), "VL",
                       np.where(kbo_sample.Zipcode.str[0].isin(["4", "5", "6", "7"]), "FR",
                       np.where(kbo_sample.Zipcode.str[0:2].between("10", "14"), "FR", "VL")))
    
    

In [17]:
if "kbo" in datasets:
    kbo_sample["street"] = np.where(kbo_sample.lg == "FR", 
                                    kbo_sample.StreetFR.fillna(kbo_sample.StreetNL),
                                    kbo_sample.StreetNL.fillna(kbo_sample.StreetFR))

    kbo_sample["city"] = np.where(kbo_sample.lg == "FR", 
                                  kbo_sample.MunicipalityFR.fillna(kbo_sample.MunicipalityNL),
                                  kbo_sample.MunicipalityNL.fillna(kbo_sample.MunicipalityFR))


In [18]:
if "kbo" in datasets:
    kbo_sample = kbo_sample.rename(columns={"Zipcode": "postcode", "HouseNumber": "housenumber"})[["street", "housenumber", "postcode", "city"]]
    
    kbo_sample.to_csv(f"{datadir}/kbo_{sample_size}.csv.gz", index=False)

In [19]:
# kbo_sample

# RRN

In [20]:
if "rrn" in datasets:
    best_RN_mapping_fn = f"{datadir}/full/3_RRN_2023Q1.zip"
    download_if_nexist(f"{bosa_mapping_url}/3_RRN_2023Q1.zip", best_RN_mapping_fn)

In [21]:
if "rrn" in datasets:
    with ZipFile(best_RN_mapping_fn, 'r') as zipObj:
        recs = []
        for f in ["STEP131_RR_B_Result.txt", "STEP131_RR_F_Result.txt", "STEP131_RR_W_Result.txt"]:
            print(f)
            for row in tqdm(zipObj.open(f) ):
                recs.append(row)

        print(f"Got {len(recs)} records, take a sample")
        random.seed(seed)
        recs= random.sample(recs, int(sample_size*1.2))

        rrn_sample = pd.DataFrame([json.loads(r) for r in recs])

STEP131_RR_B_Result.txt


487317it [00:01, 245816.24it/s]


STEP131_RR_F_Result.txt


2882103it [00:12, 229777.61it/s]


STEP131_RR_W_Result.txt


1597585it [00:06, 258858.58it/s]


Got 4967005 records, take a sample


In [22]:
if "rrn" in datasets:
    rrn_sample = rrn_sample.drop_duplicates(subset=["Snl", "Sfr", "Sde", "P", "hs"])
    rrn_sample = rrn_sample.sample(sample_size, random_state=seed)

In [23]:
if "rrn" in datasets:
    url_all_cities = "https://services.socialsecurity.be/REST/referenceData/geography/v1/countries/150/cities?pageSize=0"

    download_if_nexist(url_all_cities, f"{datadir}/full/referencedata_cities.json")

    with open(f"{datadir}/full/referencedata_cities.json", encoding='utf-8') as f:
        refdata_cities = pd.DataFrame(json.load(f)["items"])

        refdata_cities["city"]  = np.where(refdata_cities.officialLanguage=="N", refdata_cities["cityName"].apply(lambda x: x["nl"]), 
                          np.where(refdata_cities.officialLanguage=="DF", refdata_cities["cityName"].apply(lambda x: x["de"]),
                                   refdata_cities["cityName"].apply(lambda x: x["fr"])))
        refdata_cities = refdata_cities[["cityNisCode", "city"]].dropna().drop_duplicates()
        refdata_cities["cityNisCode"] = refdata_cities.cityNisCode.astype(pd.Int64Dtype()).astype(str)


In [36]:
# rrn_sample[rrn_sample.street ==""].replace("", pd.NA).Sfr.fillna(rrn_sample.Sde)

In [34]:
if "rrn" in datasets:
    rrn_sample = rrn_sample.merge(refdata_cities.rename(columns={"cityNisCode":"idM_SRC" }))
    rrn_sample["street"] = np.where(rrn_sample.R == "F", rrn_sample.Snl, rrn_sample.replace("", pd.NA).Sfr.fillna(rrn_sample.Sde).fillna(rrn_sample.Snl))

    rrn_sample = rrn_sample.rename(columns={
                        "hs":      "housenumber",
                        "POri":    "postcode", 
                        })


In [35]:
if "rrn" in datasets:
    rrn_sample[["street", "housenumber", "postcode", "city"]].to_csv(f"{datadir}/rrn_{sample_size}.csv.gz", index=False)

# Repertoire

In [77]:
# Get a sample of enterprise number from KBO open data

# 
# datasets=["rep"]

In [78]:
if "rep" in datasets:
    with ZipFile(kbo_fn) as z:
        
       # open the csv file in the dataset
        with z.open("enterprise.csv") as f:

            cbe_list_full = pd.read_csv(f,
                                        usecols  = ["EnterpriseNumber", "TypeOfEnterprise", "JuridicalForm"], 
                                        dtype=str)
    cbe_list_full

In [86]:
# sample_size=100

In [140]:
if "rep" in datasets:
    # Get a sample of CBE/KBO numbers
    cbe_list_presample = cbe_list_full[(cbe_list_full.TypeOfEnterprise == "2") & 
                                       (cbe_list_full.JuridicalForm.isin(["015", "014"]))].sample(5*sample_size, random_state=seed)
    

In [81]:
import time
def call_repertorium(cbe_number):
    url = f"https://services.socialsecurity.be/REST/employer/identification/v6/employers/search"
    
    r = requests.get(url,
        params= { "enterpriseNumber": str(cbe_number).replace(".", "")})
    
    return json.loads(r.text)

def get_repertorium_address(cbe_number):
    r = call_repertorium(cbe_number)
    
    time.sleep(0.5)
    if "identity" in r and "address" in r["identity"]:
        return r["identity"]["address"]
    else:
        return None
# get_repertorium_address(864279008)

In [141]:
# Call repertorium web service based on CBE list, until we find "sample_size*1.2" (to be sure to find sample_size unique addresses) distinct addresses
if "rep" in datasets:
    rec_with_addr=[]
    with tqdm(total=sample_size) as pbar:
        for i, rec in cbe_list_presample.reset_index(drop=True).iterrows():
            addr = get_repertorium_address(rec.EnterpriseNumber)
            if addr is not None:
                addr = {k: addr[k] for k in ['streetName','houseNumber', 'postCode', 'municipalityName']}
                if not addr in rec_with_addr :
                    #rec_with_addr.append( (rec.EnterpriseNumber, addr))
                    rec_with_addr.append(  addr)
                    pbar.update()
                else: 
                    print("skipping duplicated address")
            if len(rec_with_addr) >= sample_size:
                break

        rep_sample = pd.DataFrame(rec_with_addr)
        rep_sample

 32%|███▏      | 321/1000 [07:06<16:07,  1.43s/it]

skipping duplicated address


 95%|█████████▌| 951/1000 [21:00<01:23,  1.71s/it]

skipping duplicated address


 98%|█████████▊| 982/1000 [21:50<00:31,  1.73s/it]

skipping duplicated address


100%|█████████▉| 997/1000 [22:06<00:03,  1.08s/it]

skipping duplicated address


100%|██████████| 1000/1000 [22:12<00:00,  1.33s/it]


In [142]:
# rep_sample = pd.DataFrame(rec_with_addr)
rep_sample

,streetName,houseNumber,postCode,municipalityName
0,{'fr': 'de Burburelaan'},46,1970,{'fr': 'Wezembeek-Oppem'}
1,{'fr': 'Nachtegalenlaan'},35,1950,{'fr': 'Kraainem'}
2,{'fr': 'Rue Tilia(ST)'},12,6511,{'fr': 'Beaumont'}
3,{'fr': 'Rue du Hock(N)'},36,4121,{'fr': 'Neupré'}
4,{'nl': 'Wuustwezelseweg'},95,2990,{'nl': 'Wuustwezel'}
...,...,...,...,...
995,{'nl': 'Lichterweg'},3,2030,{'nl': 'Antwerpen'}
996,{'nl': 'Grote Markt(Kor)'},41,8500,{'nl': 'Kortrijk'}
997,{'nl': 'Heerweg-Noord'},72,9052,{'nl': 'Gent'}
998,{'nl': 'Mechelbaan'},750,2580,{'nl': 'Putte'}


In [143]:
def get_lg_pref(item, lg_order):
    for lg in lg_order:
        if lg in item:
            return item[lg]
    return None

if "rep" in datasets:
    rep_sample = rep_sample.rename(columns={"postCode": "postcode", 
                                            "houseNumber": "housenumber",
                                           })

    rep_sample["lg"] = np.where(rep_sample.postcode.str[0].isin(["2", "3", "8", "9"]), "VL",
                       np.where(rep_sample.postcode.str[0].isin(["4", "5", "6", "7"]), "FR",
                       np.where(rep_sample.postcode.str[0:2].between("10", "14"), "FR", "VL")))

    rep_sample["street"] = np.where(rep_sample.lg == "FR", 
                                    rep_sample.streetName.apply(lambda r : get_lg_pref(r, ["fr", "nl", "de"])),
                                    rep_sample.streetName.apply(lambda r : get_lg_pref(r, ["nl", "fr", "de"])))# )


    rep_sample["city"] = np.where(rep_sample.lg == "FR", 
                                  rep_sample.municipalityName.apply(lambda r : get_lg_pref(r, ["fr", "nl", "de"])),
                                  rep_sample.municipalityName.apply(lambda r : get_lg_pref(r, ["nl", "fr", "de"])))



In [144]:
if "rep" in datasets:
    rep_sample[["street", "housenumber", "postcode", "city"]].to_csv(f"{datadir}/rep_{sample_size}.csv.gz", index=False)

# Resto

In [53]:
if "resto" in datasets:
    resto_datadir = f"{datadir}/full/resto"

    resto_full = pd.concat([pd.read_csv(f) for f in glob.glob(f"{resto_datadir}/*.csv.gz")])
    resto_full

In [56]:
if "resto" in datasets:
    resto_sample = resto_full.drop("name", axis=1).drop_duplicates().sample(sample_size, random_state=seed)[["street", "housenumber", "postcode", "city"]]
                               
    resto_sample.to_csv(f"{datadir}/resto_{sample_size}.csv.gz", index=False)

In [58]:
# resto_sample